In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
# 2. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
from PIL import Image,ImageEnhance
import cv2

from google.colab.patches import cv2_imshow

In [ ]:
def find_nearest_above(my_array, target):
    diff = my_array - target
    mask = np.ma.less_equal(diff, -1)
    # We need to mask the negative differences
    # since we are looking for values above
    if np.all(mask):
        c = np.abs(diff).argmin()
        return c # returns min index of the nearest if target is greater than any value
    masked_diff = np.ma.masked_array(diff, mask)
    return masked_diff.argmin()

In [ ]:
def hist_match(original, specified):

    oldshape = original.shape
    original = original.ravel()
    specified = specified.ravel()

    # get the set of unique pixel values and their corresponding indices and counts
    s_values, bin_idx, s_counts = np.unique(original, return_inverse=True,return_counts=True)
    t_values, t_counts = np.unique(specified, return_counts=True)

    # Calculate s_k for original image
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    
    # Calculate s_k for specified image
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # Round the values
    sour = np.around(s_quantiles*255)
    temp = np.around(t_quantiles*255)
    
    # Map the rounded values
    b=[]
    for data in sour[:]:
        b.append(find_nearest_above(temp,data))
    b= np.array(b,dtype='uint8')

    return b[bin_idx].reshape(oldshape)

In [ ]:
def enhancer(im,a):
    #im = Image.open("drive/My Drive/Results 3/result0244 enhanced-a=0.6,b=0.5,g=-0.05,d=0.05 .JPG")
    enhancer = ImageEnhance.Brightness(im)
    enhanced_im = enhancer.enhance(a)
    enhanced_im.save("drive/My Drive/dummy/dummyEnhance.JPG")
    return 

In [ ]:
def brighten_YCbCr(path1,path2,alpha,beta):
    Img1 = cv2.imread(path1)
    Img2 = cv2.imread(path2)
    ycc1 = cv2.cvtColor(Img1,cv2.COLOR_BGR2YCR_CB)
    ycc2 = cv2.cvtColor(Img2,cv2.COLOR_BGR2YCR_CB)
    l = ycc1.shape[0]
    b = ycc1.shape[1]
    count = l*b
    diffv = ycc1-ycc2
    sumy = np.sum(np.square(diffv[:,:,0]))
    y = np.sqrt(sumy/count)
    p = (alpha*ycc2[:,:,0]) + (beta*y)
    ycc2[:,:,0] = p
    f = cv2.cvtColor(ycc2, cv2.COLOR_YCR_CB2BGR)
    cv2.imwrite("drive/My Drive/dummy/dummy.JPG",f)
    return 

In [ ]:
def bluefilter(orig,new):
    l = orig.shape[0]
    b = orig.shape[1]
    sumb = np.sum(orig[:,:,0])
    x = (sumb/(l*b))
    for i in range(l):
        print(i)
        for j in range(b):
            if orig[i][j][0] > (1.75*x):  
                new[i][j] = orig[i][j]
    
    return new

In [ ]:
def brighten_hsv(Img1,Img2,alpha,beta):
    hsv1 = cv2.cvtColor(Img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(Img2, cv2.COLOR_BGR2HSV)
    l = hsv1.shape[0]
    b = hsv1.shape[1]
    diffv = hsv1 - hsv2
    diffv = np.array(diffv)
    sumh = np.sum(np.square(diffv[:,:,0]))
    sums = np.sum(np.square(diffv[:,:,1]))   
    h = np.sqrt(sumh/(l*b))
    s = np.sqrt(sums/(l*b))
    p = hsv2[:,:,0] + alpha*h
    hsv2[:,:,0] = p
    q = hsv2[:,:,1] + beta*s
    hsv2[:,:,1] = q
    final = cv2.cvtColor(hsv2, cv2.COLOR_HSV2BGR)
    cv2.imwrite("drive/My Drive/dummy/dummyhsv.JPG",final)
    return

In [ ]:
p1 = "drive/My Drive/Images_for_Adarsh/DSC_6139.JPG"
p2 = "drive/My Drive/Images_for_Adarsh/DSC_0690.JPG"
brighten_YCbCr(p1,p2,1.5,2.25)
f = cv2.imread("drive/My Drive/dummy/dummy.JPG")
Img2 = cv2.imread(p2)

final = bluefilter(Img2,f)
cv2.imwrite("drive/My Drive/Results_matching Y value/0690apple(1.5,2.25).JPG",final)

In [ ]:
p1 = "drive/My Drive/Images_for_Adarsh/DSC_6139.JPG"
p2 = "drive/My Drive/Images_for_Adarsh/DSC_6181.JPG"
orig = cv2.imread(p2)
ref = cv2.imread(p1)
#f = hist_match(orig,ref)
#cv2.imwrite("drive/My Drive/dummy/dummy.JPG",f)
#brighten_YCbCr(p1,p2,1,1.5)
#f1 = cv2.imread("drive/My Drive/dummy/dummy.JPG")
ref = cv2.cvtColor(ref,cv2.COLOR_BGR2YCR_CB)
f1 = cv2.cvtColor(orig,cv2.COLOR_BGR2YCR_CB)
#brighten_hsv(ref,f1,0.3,0.5)
#f2 = cv2.imread("drive/My Drive/dummy/dummyhsv.JPG")
f2 = hist_match(f1,ref)
cv2.imwrite("drive/My Drive/dummy/dummy.JPG",f2)
f2 = cv2.imread("drive/My Drive/dummy/dummy.JPG")
f3 = cv2.cvtColor(f2,cv2.COLOR_BGR2YCR_CB)
f3[:,:,1] = f1[:,:,1]
f3[:,:,2] = f1[:,:,2]
f3 = cv2.cvtColor(f3,cv2.COLOR_YCR_CB2BGR)
final = bluefilter(orig,f3)
cv2.imwrite("drive/My Drive/Results-Y value matching+histogram matching/6181*.JPG",final)

In [ ]:
p1 = "drive/My Drive/Images_for_Adarsh/DSC_6139.JPG"
p2 = "drive/My Drive/YCrCb Results/6138(1,0.75).JPG"
Img1 = cv2.imread(p1)
Img2 = cv2.imread(p2)

f = hist_match(Img2,Img1)
final = bluefilter(Img2,f)
cv2.imwrite("drive/My Drive/YCrCb+histogram_matching/0690**.JPG",final)